In [ ]:
import torch
import pandas as pd
import numpy as np

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")
dataset["train"].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [ ]:
from transformers import BertTokenizerFast

tokenizer: BertTokenizerFast = BertTokenizerFast.from_pretrained("bert-base-uncased")


def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
    )


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

<a id='trainer'></a>

In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoConfig,
    BertForSequenceClassification,
)

config = AutoConfig.from_pretrained(
    "bert-base-uncased",
    # num_labels=2,
)
# model = AutoModelForSequenceClassification.from_config(config)
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="bert_pretrained",
    evaluation_strategy="epoch",
    use_mps_device=torch.backends.mps.is_available(),
    optim="adamw_torch",
)

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.348700,0.236343,0.898000
2,0.187800,0.320808,0.922000
3,0.067700,0.406377,0.924000


TrainOutput(global_step=3750, training_loss=0.2256259084065755, metrics={'train_runtime': 1022.0878, 'train_samples_per_second': 29.352, 'train_steps_per_second': 3.669, 'total_flos': 7893331660800000.0, 'train_loss': 0.2256259084065755, 'epoch': 3.0})

In [ ]:
trainer.save_model()
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.3858458995819092,
 'eval_accuracy': 0.9256,
 'eval_runtime': 293.7491,
 'eval_samples_per_second': 85.107,
 'eval_steps_per_second': 10.638,
 'epoch': 3.0}